In [66]:
import pandas as pd
import sqlite3

In [67]:
conn = sqlite3.connect('pets.db')
c = conn.cursor()

In [68]:
!ls

3_2_sql_joins.md            dogs.db
Untitled.ipynb              owners.db
cat_dog_owner_sqlite3.ipynb pets.db
cats.db


In [69]:
c.execute ('''select name from sqlite_master where type = 'table';''').fetchall()

[('owners',), ('dogs',), ('cats',)]

In [70]:
c.execute('''SELECT *
             FROM sqlite_master
             WHERE type='table';''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head()

,type,name,tbl_name,rootpage,sql
0,table,owners,owners,2,"CREATE TABLE owners(\n ""Name"" TEXT,\n ""Phone..."
1,table,dogs,dogs,3,"CREATE TABLE dogs(\n ""Name"" TEXT,\n ""Breed"" ..."
2,table,cats,cats,4,"CREATE TABLE cats(\n ""Name"" TEXT,\n ""Breed"" ..."


In [71]:
#c.execute('''SELECT *
             #FROM dogs;''')
# df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
#df.columns = [i[0] for i in c.description]
#df.head()

In [72]:
 c.execute('''SELECT *
              FROM cats;''')
 df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
 df.columns = [i[0] for i in c.description]
 df.head(50)

,Name,Breed,House_Number,Owner_ID
0,Bell,Siamese,1,4
1,Jackson,Balinese,2,3
2,Precious,Himalayan,3,4
3,Rocky,Egyptian Mau,4,2
4,Samson,Javanese,5,1


In [73]:
c.execute('''SELECT *
             FROM dogs;''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)

,Name,Breed,House_Number,Owner_ID
0,Rex,Chihuahua,1,3
1,Clifford,German Shepherd,2,2
2,Lucky,Daschund,3,1
3,Bobo,Shih-Tzu,2,4
4,Buddy,Golden Retriever,4,2
5,Leo,English Bulldog,1,2


In [86]:
c.execute('''SELECT *
             FROM owners;''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(10)

,Name,Phone,Address,ID
0,Josh Daniell,3132879573,1440 G st.,1
1,Alison Peebles Madigan,2147098190,None,2
2,Avi Flombaum,4698780125,1776 New York Ave,3
3,Justin Bieber,4103810987,None,4


In [75]:
#A. Write a SQL query that can grab the names of all pets by each owner (because we want the pets' name - UNION)

c.execute('''SELECT *
             FROM 
                 (
                 SELECT owners.Name as ownerName, dogs.Name as petName
                 FROM owners JOIN dogs ON owners.ID = dogs.Owner_ID

                 UNION

                 SELECT owners.Name as ownerName, cats.Name as petName
                 FROM owners JOIN cats ON owners.ID = cats.Owner_ID
                 )
              ORDER BY ownerName;
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)


,ownerName,petName
0,Alison Peebles Madigan,Buddy
1,Alison Peebles Madigan,Clifford
2,Alison Peebles Madigan,Leo
3,Alison Peebles Madigan,Rocky
4,Avi Flombaum,Jackson
5,Avi Flombaum,Rex
6,Josh Daniell,Lucky
7,Josh Daniell,Samson
8,Justin Bieber,Bell
9,Justin Bieber,Bobo


In [76]:
#A. Write a SQL query that can grab the names of all pets owned by the founder of Flatiron School, Avi Flombaum

c.execute('''SELECT *
             FROM 
                 (
                 SELECT owners.Name as ownerName, dogs.Name as petName
                 FROM owners JOIN dogs ON owners.ID = dogs.Owner_ID

                 UNION

                 SELECT owners.Name as ownerName, cats.Name as petName
                 FROM owners JOIN cats ON owners.ID = cats.Owner_ID
                 )
              WHERE ownerName = "Avi Flombaum"
              ORDER BY ownerName;
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)

,ownerName,petName
0,Avi Flombaum,Jackson
1,Avi Flombaum,Rex


In [77]:
#B. Which Owner have the most number of pets and how many? (because we just need the number of pets - JOIN)
c.execute('''SELECT owners.Name, (count(dogs.Name) + count(cats.Name)) as total_pets
             FROM owners JOIN dogs ON owners.ID = dogs.Owner_ID
                         JOIN cats ON owners.ID = cats.Owner_ID
             GROUP BY  owners.Name
             ORDER BY total_pets DESC
             --LIMIT 1; -- or fetchone()
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head()

,Name,total_pets
0,Alison Peebles Madigan,6
1,Justin Bieber,4
2,Avi Flombaum,2
3,Josh Daniell,2


In [78]:
#B. Which Owner have the most number of pets and how many? (we use MAX)
c.execute('''SELECT Name, max(total_pets)
             FROM 
                 (
                 SELECT owners.Name, (count(dogs.Name) + count(cats.Name)) as total_pets
                 FROM owners JOIN dogs ON owners.ID = dogs.Owner_ID
                             JOIN cats ON owners.ID = cats.Owner_ID
                 GROUP BY  owners.Name
                 )
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head()

,Name,max(total_pets)
0,Alison Peebles Madigan,6


In [79]:
#B. How many pets each House_Number contains, and which one have the most of pets? 
# (BECAUSE we just need total pets in each House_Number - GROUP BY)
c.execute('''SELECT House_Number, count(Name) as Total_Pets
             FROM
                 (
                 SELECT Name, House_Number, Owner_ID
                 FROM dogs
                 UNION
                 SELECT Name, House_Number, Owner_ID
                 FROM cats
                 )
             GROUP BY House_Number;
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)

,House_Number,Total_Pets
0,1,3
1,2,3
2,3,2
3,4,2
4,5,1


In [80]:
#B. How many pets each House_Number contains? (BECAUSE we want to show the name of pets in each House_Number - ORDER BY)
c.execute('''SELECT *
             FROM
                 (
                 SELECT Name, House_Number, Owner_ID
                 FROM dogs
                 
                 UNION
                 
                 SELECT Name, House_Number, Owner_ID
                 FROM cats
                 )
              ORDER BY House_Number DESC;
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)

,Name,House_Number,Owner_ID
0,Samson,5,1
1,Buddy,4,2
2,Rocky,4,2
3,Lucky,3,1
4,Precious,3,4
5,Bobo,2,4
6,Clifford,2,2
7,Jackson,2,3
8,Bell,1,4
9,Leo,1,2


In [81]:
# This one is not right because there are 2 House_Number have the same max total pets are 3 in each.
#B. Which House_Number contains the most number of pets in it and how many?
c.execute('''SELECT House_Number, count(*) as Total_Pets
             FROM
                 (
                 SELECT *
                 FROM dogs
                 UNION
                 SELECT *
                 FROM cats
                 )
             GROUP BY House_Number
             ORDER BY Total_Pets DESC
             LIMIT 1;
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(10)

,House_Number,Total_Pets
0,1,3


In [82]:
# Try to use max to prove that max won't show more than 1 row. we better use the above solution.
c.execute('''SELECT House_Number, max(Total_Pets)
             FROM
                 (
                 SELECT House_Number, count(*) as Total_Pets
                 FROM
                     (
                     SELECT *
                     FROM dogs

                     UNION

                     SELECT *
                     FROM cats
                     )
                 GROUP BY House_Number
                 );
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head()

,House_Number,max(Total_Pets)
0,1,3


In [83]:
#B. Which House_Number contains the most number of pets in it and how many?
c.execute('''SELECT *
             FROM
                 (
                 SELECT *
                 FROM dogs
                 UNION
                 SELECT *
                 FROM cats
                 )
             --GROUP BY House_Number
             --ORDER BY Total_Pets DESC
             --LIMIT 1;
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)

,Name,Breed,House_Number,Owner_ID
0,Bell,Siamese,1,4
1,Bobo,Shih-Tzu,2,4
2,Buddy,Golden Retriever,4,2
3,Clifford,German Shepherd,2,2
4,Jackson,Balinese,2,3
5,Leo,English Bulldog,1,2
6,Lucky,Daschund,3,1
7,Precious,Himalayan,3,4
8,Rex,Chihuahua,1,3
9,Rocky,Egyptian Mau,4,2


In [88]:
#The daycare realized that House 4 needs the owners of its pets to update their addresses. 
#Grab the names and phone numbers of owners with pets in house 4 that do not currently have an address.
c.execute('''SELECT *
             FROM
                 (
                 SELECT owners.Name as Name, Address, Phone
                 FROM dogs JOIN owners ON dogs.Owner_ID = owners.ID
                 WHERE House_Number = 4 AND  Address = 'None'
                 UNION
                 SELECT owners.Name as Name, Address, Phone
                 FROM cats JOIN owners ON cats.Owner_ID = owners.ID
                 WHERE House_Number = 4 AND  Address = 'None'
                 );
          ''')
df = pd.DataFrame(c.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in c.description]
df.head(50)

,Name,Address,Phone
0,Alison Peebles Madigan,None,2147098190
